In [1]:
import sys
sys.path.append('../')
import tabularepimdl as tepi

import pandas as pd
import numpy as np

In [2]:
hh_pop = pd.DataFrame({
    'InfState':pd.Categorical(["S"]*4+["I"],  categories=['S','I','R']),
    'HH_Number': list(range(4))+[1],
    'N': [4, 3, 2, 5, 1],
    'T':0
})
hh_pop

,InfState,HH_Number,N,T
0,S,0,4,0
1,S,1,3,0
2,S,2,2,0
3,S,3,5,0
4,I,1,1,0


In [18]:
hh_pop['InfState'].dtype

CategoricalDtype(categories=['S', 'I', 'R'], ordered=False, categories_dtype=object)

The EnvironmentalTransmission process steps:
1. get the number of S folks out of population.
2. probability of infection (prI) = 1 - np.exp(-dt*in_beta)
3. calculate deltas using S folks and prI
4. update population - subtract deltas from the S folks and add deltas to the I folks.

In [3]:
env_infect = tepi.EnvironmentalTransmission(beta=0.2/5, inf_col="InfState", trait_col="HH_Number")

In [4]:
return_deltas = env_infect.get_deltas(hh_pop)
return_deltas

env combined deltas are
   InfState  HH_Number         N  T
0        S          0 -0.156842  0
1        S          1 -0.117632  0
2        S          2 -0.078421  0
3        S          3 -0.196053  0
0        I          0  0.156842  0
1        I          1  0.117632  0
2        I          2  0.078421  0
3        I          3  0.196053  0


,InfState,HH_Number,N,T
0,S,0,-0.156842,0
1,S,1,-0.117632,0
2,S,2,-0.078421,0
3,S,3,-0.196053,0
4,I,0,0.156842,0
5,I,1,0.117632,0
6,I,2,0.078421,0
7,I,3,0.196053,0


In [19]:
return_deltas['InfState'].dtype

dtype('O')

In [5]:
#New S and I folks distribution in the population
pd.concat([return_deltas, hh_pop]).groupby(['InfState', 'HH_Number']).agg({'N': 'sum', 'T': 'max'}).reset_index()

,InfState,HH_Number,N,T
0,I,0,0.156842,0
1,I,1,1.117632,0
2,I,2,0.078421,0
3,I,3,0.196053,0
4,S,0,3.843158,0
5,S,1,2.882368,0
6,S,2,1.921579,0
7,S,3,4.803947,0


In [6]:
env_infect.to_yaml()

{'tabularepimdl.EnvironmentalTransmission': {'stochastic': False,
  'beta': 0.04,
  'inf_col': 'InfState',
  'trait_col': 'HH_Number',
  's_st': 'S',
  'i_st': 'I',
  'inf_to': 'I'}}

In [17]:
env_infect_stoch = tepi.EnvironmentalTransmission(beta=0.2/5, inf_col="InfState", trait_col="HH_Number", stochastic=True)
return_deltas_stoch = env_infect_stoch.get_deltas(hh_pop)
return_deltas_stoch

env combined deltas are
   InfState  HH_Number  N  T
0        S          0  0  0
1        S          1 -1  0
2        S          2  0  0
3        S          3  0  0
0        I          0  0  0
1        I          1  1  0
2        I          2  0  0
3        I          3  0  0


,InfState,HH_Number,N,T
0,S,1,-1,0
1,I,1,1,0
